In [14]:
import cv2
import numpy as np

def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    height, width = edges.shape[:2]
    mask = np.zeros_like(edges)
    region_of_interest_vertices = np.array([[(200, height), (800, 350), (1200, height)]], dtype=np.int32)
    cv2.fillPoly(mask, region_of_interest_vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked_edges, 2, np.pi / 180, 100, np.array([]), minLineLength=40, maxLineGap=5)

    left_lines = []
    right_lines = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - slope * x1

            if slope < -0.2:
                left_lines.append((slope, intercept))
            elif slope > 0.2:
                right_lines.append((slope, intercept))

    line_image = np.zeros_like(frame)

    if left_lines:
        left_avg_slope, left_avg_intercept = np.average(left_lines, axis=0)
        draw_arrow(line_image, left_avg_slope, left_avg_intercept, color=(255, 0, 0))

    if right_lines:
        right_avg_slope, right_avg_intercept = np.average(right_lines, axis=0)
        draw_arrow(line_image, right_avg_slope, right_avg_intercept, color=(0, 0, 255))

    result = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    return result

def draw_arrow(image, slope, intercept, color=(255, 0, 0), thickness=10, arrow_length=40, arrow_angle=np.pi / 6):
    y1 = image.shape[0]
    y2 = int(y1 * 3 / 5)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)

    cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    angle = np.arctan2(y2 - y1, x2 - x1)

    x_arrow = int(x2 - arrow_length * np.cos(angle - arrow_angle))
    y_arrow = int(y2 - arrow_length * np.sin(angle - arrow_angle))
    cv2.line(image, (x2, y2), (x_arrow, y_arrow), color, thickness)

    x_arrow = int(x2 - arrow_length * np.cos(angle + arrow_angle))
    y_arrow = int(y2 - arrow_length * np.sin(angle + arrow_angle))
    cv2.line(image, (x2, y2), (x_arrow, y_arrow), color, thickness)

cap = cv2.VideoCapture("test_video.mp4")

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    result = process_frame(frame)
    cv2.imshow("result", result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()